In [ ]:
import subprocess
import json

In [ ]:
php = 'php.exe ./hd-wallet-derive/hd-wallet-derive.php'
mnem = "silver what pig slice hint goose chapter clean shove brand dismiss laptop"

In [ ]:
def getKeys(cmd): 
    p = subprocess.Popen(cmd, 
        stdout=subprocess.PIPE, shell=False) 
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [ ]:
results = []
coins = ['BTC-test', 'ETH']
for coin in coins:
    cmd = f'{php} -g --mnemonic="{mnem}" --cols=path,address,privkey,pubkey --coin={coin} --format=json'
    keys = getKeys(cmd)
    for key in keys:
        results.append({"Coin": coin, "Address": key["address"], "PrivKey": key["privkey"], "PubKey": key["pubkey"]})

In [ ]:
print(results)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df

In [ ]:
from bit import wif_to_key
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [ ]:
def getbal(row):
    privKey = row["PrivKey"]
    address = row["Address"]
    coin = row["Coin"]
    print(coin, address)
    if coin == 'BTC-test':
        key = wif_to_key(privKey)
        bal = key.get_balance('btc')
    else:
        bal = w3.eth.getBalance(address)
    return bal
    
df["bal"] = df.apply(getbal, axis=1)

In [ ]:
df2 = df[['Coin', 'Address', 'bal']]
df2